### Démarrage de la session Spark

In [1]:
# L'exécution de cette cellule démarre l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1681301526557_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1681301526557_0002,pyspark,idle,Link,Link,None,✔


###  Installation des packages

Les packages nécessaires ont été installé via l'étape de bootstrap à l'instanciation du serveur.

### Import des librairies

In [3]:
import pandas as pd
import numpy as np

from PIL import Image
from pathlib import Path
import io
import os
#import cv2

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

from sklearn import decomposition
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Définition des PATH pour charger les images et enregistrer les résultats

In [4]:
PATH = 's3://projetopenclassrooms'
PATH_Data = PATH+'/Test/'
PATH_Result = PATH+'/Results/'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://projetopenclassrooms
PATH_Data:   s3://projetopenclassrooms/Test/
PATH_Result: s3://projetopenclassrooms/Results/

### Chargement des données

In [5]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://projetopencl...|2022-09-28 18:03:57|  7353|[FF D8 FF E0 00 1...|
|s3://projetopencl...|2022-09-28 18:03:58|  7350|[FF D8 FF E0 00 1...|
|s3://projetopencl...|2022-09-28 18:03:58|  7349|[FF D8 FF E0 00 1...|
|s3://projetopencl...|2022-09-28 18:03:57|  7348|[FF D8 FF E0 00 1...|
|s3://projetopencl...|2022-09-28 18:04:29|  7328|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

In [7]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+-------------------------------------------------------+----------+
|path                                                   |label     |
+-------------------------------------------------------+----------+
|s3://projetopenclassrooms/Test/Watermelon/r_106_100.jpg|Watermelon|
|s3://projetopenclassrooms/Test/Watermelon/r_109_100.jpg|Watermelon|
|s3://projetopenclassrooms/Test/Watermelon/r_108_100.jpg|Watermelon|
|s3://projetopenclassrooms/Test/Watermelon/r_107_100.jpg|Watermelon|
|s3://projetopenclassrooms/Test/Watermelon/r_95_100.jpg |Watermelon|
+-------------------------------------------------------+----------+
only showing top 5 rows

None

### Préparation du modèle

In [8]:
model = ResNet50(weights='imagenet',include_top=False,pooling="avg")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-1].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                           

In [12]:
def model_fn():
    """
    Returns a ResNet50 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = ResNet50(weights='imagenet',include_top=False,pooling="avg")
    
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-1].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Définition du processus de chargement des images et application de leur featurisation à travers l'utilisation de pandas UDF

In [13]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt/yarn/usercache/livy/appcache/application_1681301526557_0002/container_1681301526557_0002_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:398: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

### Exécutions des actions d'extractions de features

In [14]:
features_df = images.repartition(20).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://projetopenclassrooms/Results/

In [17]:
features_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Chargement des données enregistrées et validation du résultat

In [18]:
df = pd.read_parquet(PATH_Result, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                path  ...                                           features
0  s3://projetopenclassrooms/Test/Watermelon/r_10...  ...  [0.0, 0.024642652, 0.11504222, 0.0, 0.20724857...
1  s3://projetopenclassrooms/Test/Watermelon/r_68...  ...  [0.4324392, 0.07219571, 0.046798065, 0.0, 0.13...
2  s3://projetopenclassrooms/Test/Pineapple Mini/...  ...  [0.09303992, 0.038460862, 1.5252075, 0.0021120...
3  s3://projetopenclassrooms/Test/Pineapple Mini/...  ...  [0.0493772, 0.029517485, 1.8227782, 0.0, 0.042...
4  s3://projetopenclassrooms/Test/Pineapple Mini/...  ...  [0.07643009, 0.32158664, 1.0555596, 6.595193e-...

[5 rows x 3 columns]

In [20]:
df.loc[0,'features'].shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(2048,)

In [21]:
df.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(16446, 3)

### Dimensional reduction with PCA

In [22]:
for i in range(0,len(df["features"].values[0])):
    def format_features(val):
        x=val[i]
        return x
    df["feature{}".format(i+1)]=df["features"].apply(format_features)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
features =df.drop(columns=['path', 'label', 'features'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
features.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   feature1  feature2  feature3  ...  feature2046  feature2047  feature2048
0  0.000000  0.024643  0.115042  ...     0.551871     0.033680     0.000000
1  0.432439  0.072196  0.046798  ...     0.160982     0.062535     0.015878
2  0.093040  0.038461  1.525208  ...     0.012783     0.016450     0.099365
3  0.049377  0.029517  1.822778  ...     0.000000     0.132997     0.207595
4  0.076430  0.321587  1.055560  ...     0.000000     0.002809     0.165099

[5 rows x 2048 columns]

In [29]:
PATH_Result_PCA = PATH+'/PCA/'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
print(PATH_Result_PCA)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://projetopenclassrooms/PCA/

In [26]:
#spark.createDataFrame(features).write.mode("overwrite").parquet(PATH_Result_PCA)

An error was encountered:
Session 0 did not reach idle status in time. Current status is busy.


In [ ]:
#features = pd.read_parquet(PATH_Result_PCA, engine='pyarrow')

In [ ]:
#features.head()

In [25]:
scaler=StandardScaler() 
features_scaled=scaler.fit_transform(features)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
pca = decomposition.PCA(n_components=min(features.shape[1],
                                         features.shape[0])).fit(features_scaled)
nbr_pca=0
scree = pca.explained_variance_ratio_
for i in range(features.shape[1]):
    a = scree.cumsum()[i]
    if a >= 0.8:
        print("{} principal components explaines  80% of the total variance".format(i))
        print("Sum of variance explained :{}%".format(round(a*100,2)))
        nbr_pca=i
        break
pca = decomposition.PCA(n_components=nbr_pca)# apply PCA

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

135 principal components explaines  80% of the total variance
Sum of variance explained :80.09%

In [27]:
df_pca=pca.fit_transform(features_scaled)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
df_pca=pd.DataFrame(df_pca, columns=["PCA n°{}".format(i+1) for i in range(0,nbr_pca) ])
df_pca["label"]=df["label"].values
df_pca.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     PCA n?1   PCA n?2    PCA n?3  ...  PCA n?134  PCA n?135           label
0   4.013030 -1.719506   8.731134  ...  -0.764047   0.347654      Watermelon
1  11.409085 -0.512406   4.394932  ...   0.761277  -3.587806      Watermelon
2  18.010042  4.649156  34.132473  ...  -0.133073  -0.186032  Pineapple Mini
3  19.968901  3.255347  30.935589  ...  -0.319296  -0.720151  Pineapple Mini
4  20.296071  4.910512  34.431994  ...   0.272681   2.092303  Pineapple Mini

[5 rows x 136 columns]

In [31]:
spark.createDataFrame(df_pca).write.mode("overwrite").parquet(PATH_Result_PCA)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df_pca = pd.read_parquet(PATH_Result_PCA, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
df_pca.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     PCA n?1   PCA n?2    PCA n?3  ...  PCA n?134  PCA n?135           label
0   4.013030 -1.719506   8.731134  ...  -0.764047   0.347654      Watermelon
1  11.409085 -0.512406   4.394932  ...   0.761277  -3.587806      Watermelon
2  18.010042  4.649156  34.132473  ...  -0.133073  -0.186032  Pineapple Mini
3  19.968901  3.255347  30.935589  ...  -0.319296  -0.720151  Pineapple Mini
4  20.296071  4.910512  34.431994  ...   0.272681   2.092303  Pineapple Mini

[5 rows x 136 columns]